In [1]:
import keras
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
# import keras_metrics as km
from keras import regularizers, optimizers
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import accuracy_score
import math

In [2]:
data_dir = "/content/drive/MyDrive/video_data"
img_height , img_width = 64, 64
seq_len = 16

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
classes = ["fall", "fall_down", "grab", "swallow", "walk","sit"]

In [5]:
def frames_extraction(video_path):
    frames_list = []
     
    vidcap = cv2.VideoCapture(video_path)
    total_frames = vidcap.get(cv2.CAP_PROP_FRAME_COUNT)

    frames_step = total_frames//seq_len
 
    for j in range(seq_len):
        #here, we set the parameter 1 which is the frame number to the frame (i*frames_step)
        vidcap.set(1,j*frames_step)
        success,image = vidcap.read()  
        #save your image
        if success:
          image = cv2.resize(image, (img_height, img_width))
          frames_list.append(image)
        else:
          print("Defected frame {}".format(video_path))
          break

    return frames_list

In [6]:
def create_data(input_dir):
    X = []
    Y = []
     
    classes_list = os.listdir(input_dir)
     
    for c in classes_list:
        print(c)
        files_list = os.listdir(os.path.join(input_dir, c))
        for f in files_list:
            frames = frames_extraction(os.path.join(os.path.join(input_dir, c), f))
            if len(frames) == seq_len:
                X.append(frames)
             
                y = [0]*len(classes)
                y[classes.index(c)] = 1
                Y.append(y)
     
    X = np.asarray(X)
    Y = np.asarray(Y)
    return X, Y

In [7]:
X, Y = create_data(data_dir)
 
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, shuffle=True, random_state=0)

walk
Defected frame /content/drive/MyDrive/video_data/walk/135.mov
Defected frame /content/drive/MyDrive/video_data/walk/143.mov
Defected frame /content/drive/MyDrive/video_data/walk/101.mov
fall_down
Defected frame /content/drive/MyDrive/video_data/fall_down/145.mov
Defected frame /content/drive/MyDrive/video_data/fall_down/143.mov
Defected frame /content/drive/MyDrive/video_data/fall_down/172.mov
Defected frame /content/drive/MyDrive/video_data/fall_down/21.mov
Defected frame /content/drive/MyDrive/video_data/fall_down/26.mov
Defected frame /content/drive/MyDrive/video_data/fall_down/46.mov
Defected frame /content/drive/MyDrive/video_data/fall_down/52.mov
Defected frame /content/drive/MyDrive/video_data/fall_down/57.mov
Defected frame /content/drive/MyDrive/video_data/fall_down/81.mov
swallow
Defected frame /content/drive/MyDrive/video_data/swallow/47.mov
Defected frame /content/drive/MyDrive/video_data/swallow/70.mov
Defected frame /content/drive/MyDrive/video_data/swallow/79_crop (

In [8]:
model = Sequential()
model.add(ConvLSTM2D(filters = 64, kernel_size = (3, 3), return_sequences = True, padding = 'same', data_format = "channels_last", input_shape = (seq_len, img_height, img_width, 3)))
model.add(ConvLSTM2D(filters = 64, kernel_size = (3, 3), return_sequences = True, padding = 'same', data_format = "channels_last"))
model.add(BatchNormalization())
model.add(Dropout(0.5))


model.add(ConvLSTM2D(filters = 32, kernel_size = (3, 3), return_sequences = True, padding = 'same', data_format = "channels_last"))
model.add(ConvLSTM2D(filters = 32, kernel_size = (3, 3), return_sequences = False, padding = 'same', data_format = "channels_last"))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(6, activation = "softmax"))
 
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d (ConvLSTM2D)    (None, 16, 64, 64, 64)    154624    
_________________________________________________________________
conv_lst_m2d_1 (ConvLSTM2D)  (None, 16, 64, 64, 64)    295168    
_________________________________________________________________
batch_normalization (BatchNo (None, 16, 64, 64, 64)    256       
_________________________________________________________________
dropout (Dropout)            (None, 16, 64, 64, 64)    0         
_________________________________________________________________
conv_lst_m2d_2 (ConvLSTM2D)  (None, 16, 64, 64, 32)    110720    
_________________________________________________________________
conv_lst_m2d_3 (ConvLSTM2D)  (None, 64, 64, 32)        73856     
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 64, 32)        1

In [9]:
def schedular(epoch):
  if epoch < 10:
    return 0.001
  else:
    return 0.001 * math.exp(0.1*(10-epoch))

In [ ]:
opt = optimizers.Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])
 
earlystop = EarlyStopping(patience=7)
callbacks = tf.keras.callbacks.LearningRateScheduler(schedular)
 
history = model.fit(x = X_train, y = y_train, epochs=20, batch_size = 32, shuffle=True, validation_split=0.2, callbacks=[callbacks])

import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train', 'val','accuracy','val_accuracy'])
plt.show()


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/20
19/19 [==============================] - 2098s 109s/step - loss: 10.6885 - accuracy: 0.2091 - val_loss: 17.9129 - val_accuracy: 0.1141
Epoch 2/20
19/19 [==============================] - 2029s 107s/step - loss: 2.0375 - accuracy: 0.3538 - val_loss: 2.9218 - val_accuracy: 0.2550
Epoch 3/20
19/19 [==============================] - 2024s 107s/step - loss: 1.6324 - accuracy: 0.3619 - val_loss: 8.6866 - val_accuracy: 0.3087
Epoch 4/20
19/19 [==============================] - 2026s 107s/step - loss: 1.6528 - accuracy: 0.3513 - val_loss: 5.3754 - val_accuracy: 0.3221
Epoch 5/20
19/19 [==============================] - 2044s 108s/step - loss: 1.6354 - accuracy: 0.3425 - val_loss: 1.5996 - val_accuracy: 0.3087
Epoch 6/20
19/19 [==============================] - 2058s 109s/step - loss: 1.6209 - accuracy: 0.3584 - val_loss: 1.6146 - val_accuracy: 0.3691
Epoch 7/20
19/19 [==============================] - 2051s 108s/step - loss: 1.4903 - accuracy: 0.3985 - val_loss: 2.0842 - val_accurac

In [13]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_test) 
y_pred = np.argmax(y_pred, axis = 1)
# y_test = np.argmax(y_test, axis = 1)
 
print(classification_report(y_test, y_pred))

print('accuracy score : {}'.format(accuracy_score(y_pred, y_test)))


              precision    recall  f1-score   support

           0       1.00      0.45      0.62        20
           1       0.38      0.25      0.30        48
           2       0.59      0.69      0.64        32
           3       0.57      0.61      0.59        33
           4       0.35      0.71      0.47        35
           5       1.00      0.11      0.19        19

    accuracy                           0.48       187
   macro avg       0.65      0.47      0.47       187
weighted avg       0.57      0.48      0.46       187

accuracy score : 0.48128342245989303
